In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import models
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense, ConvLSTM2D, MultiHeadAttention, Flatten, MaxPooling3D, BatchNormalization, Dropout
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import os
import random
from pathlib import Path
import imageio
from IPython import display
from urllib import request
from tensorflow_docs.vis import embed


In [ ]:
# RESNET pretrained
pretrained_model = tf.keras.applications.ResNet50(include_top=False,
                                                  input_shape=(224, 224, 3),
                                                  pooling='avg')

# buang avg pool layer
gap_layer_name = 'avg_pool'
gap_layer_index = None

for i, layer in enumerate(pretrained_model.layers):
    if layer.name == gap_layer_name:
        gap_layer_index = i
        break


if gap_layer_index is not None:
    model_without_gap = tf.keras.Model(inputs=pretrained_model.input, outputs=pretrained_model.layers[gap_layer_index - 1].output)

else:
    print("GAP layer not found.")

In [ ]:
video_input_shape = (20, 224, 224, 3)

video_input = tf.keras.Input(shape=video_input_shape)
time_distributed = TimeDistributed(model_without_gap)(video_input)

conv_lstm_output = ConvLSTM2D(filters=256, kernel_size=(3, 3), padding="same", return_sequences=True)(time_distributed)

mhsa_output = MultiHeadAttention(num_heads=8, key_dim=256, value_dim=256)(conv_lstm_output, conv_lstm_output)
# attention_time = TimeDistributed(tf.keras.layers.Lambda(lambda x: x))(mhsa_output)

conv_lstm_output2 = ConvLSTM2D(filters=256, kernel_size=(3, 3), padding="same", return_sequences=True)(mhsa_output)

maxpool3d = MaxPooling3D(pool_size=(1,2,2))(conv_lstm_output2)

flattened = Flatten()(maxpool3d)

dense_1 = Dense(1000, activation="relu")(flattened)
dense_2 = Dense(256, activation="relu")(dense_1)
dense_3 = Dense(10, activation="relu")(dense_2)
dense_4 = Dense(2, activation="softmax")(dense_3)

# Create the model
model_no_pretrain = tf.keras.Model(inputs=video_input, outputs=dense_4)

model_no_pretrain.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 20, 224, 224, 3)]    0         []                            
                                                                                                  
 time_distributed_2 (TimeDi  (None, 20, 7, 7, 2048)       2358771   ['input_6[0][0]']             
 stributed)                                               2                                       
                                                                                                  
 conv_lstm2d_4 (ConvLSTM2D)  (None, 20, 7, 7, 256)        2123468   ['time_distributed_2[0][0]']  
                                                          8                                       
                                                                                            

In [ ]:
epochs = 20
lr = 1e-6

In [ ]:
model_no_pretrain.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr),
              metrics=['accuracy', AUC(multi_label=False)])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_no_pretrain.load_weights('/content/drive/MyDrive/model_no_pt_weights.h5')

In [ ]:
def video2frame(filepath, resize=(224,224)):

    cap = cv2.VideoCapture(filepath)
    # num of frames
    len_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # print(len_frames)
    len_frames = int(len_frames)
    frames = []
    try:
        for i in range(len_frames):
            _, frame = cap.read()
            frame = cv2.resize(frame, resize, interpolation=cv2.INTER_AREA)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = np.reshape(frame, (224, 224, 3))
            frames.append(frame)
    except:
        print("error: ", filepath, len_frames, i)
    finally:
        frames = np.array(frames)
        cap.release()

    flows = calc_optical_flow(frames)

    result = np.zeros((len(flows), 224, 224, 5))
    result[..., :3] = frames
    result[..., 3:] = flows

    frames = uniform_sampling(result, 20)
    return frames


def frame_difference(frames):
    num_frames = len(frames)
    out = []
    for i in range(num_frames - 1):
        out.append(cv2.subtract(frames[i + 1], frames[i]))

    return np.array(out)


def uniform_sampling(frames, target_frames=10):
    num_frames = len(frames)
    skip_frames = num_frames//target_frames
    out = []

    for i in range(target_frames):
        out.append(frames[i * skip_frames])

    return np.array(out)


def flow_to_color(video):
    rgb_flows = []

    cap = cv2.VideoCapture(video)

    ret, first_frame = cap.read()

    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

    # Creates an image filled with zero intensities with the same dimensions as the frame
    mask = np.zeros_like(first_frame)

    # Sets image saturation to maximum
    mask[..., 1] = 255

    while (cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculates dense optical flow by Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray,
                                           None,
                                           0.5, 3, 15, 3, 5, 1.2, 0)

        # Computes the magnitude and angle of the 2D vectors
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Sets image hue according to the optical flow direction
        mask[..., 0] = angle * 180 / np.pi / 2

        # Sets image value according to the optical flow magnitude (normalized)
        mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

        # Converts HSV to RGB (BGR) color representation
        rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)

        resized_rgb = cv2.resize(rgb, (224, 224), interpolation=cv2.INTER_AREA)
        rgb_flows.append(resized_rgb)

        prev_gray = gray

        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

    cap.release()
    # rgb_flows.append(np.zeros((224, 224, 3)))
    # cv2.destroyAllWindows()

    return np.array(rgb_flows)



def Save2Npy(file_dir, save_dir):
    """Transfer all the videos and save them into specified directory
    Args:
        file_dir: source folder of target videos
        save_dir: destination folder of output .npy files
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    # List the files
    videos = os.listdir(file_dir)
    for v in tqdm(videos):
        # Split video name
        video_name = v.split('.')[0]
        # Get src
        video_path = os.path.join(file_dir, v)
        # Get dest
        save_path = os.path.join(save_dir, video_name + '.npy')
        # Load and preprocess video
        data = video2frame(video_path, resize=(224, 224))
        data = np.uint8(data)
        # Save as .npy file
        np.save(save_path, data)

    return None

def Save2Npy2(file_dir, save_dir):
    """Transfer all the videos and save them into specified directory
    Args:
        file_dir: source folder of target videos
        save_dir: destination folder of output .npy files
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    # List the files
    videos = os.listdir(file_dir)
    for v in tqdm(videos):
        # Split video name
        video_name = v.split('.')[0]
        # Get src
        video_path = os.path.join(file_dir, v)
        # Get dest
        save_path = os.path.join(save_dir, video_name + '.npy')
        # Load and preprocess video
        data = flow_to_color(video_path)
        data = np.uint8(data)
        # Save as .npy file
        np.save(save_path, data)

    return None

def calc_optical_flow(frames):
    gray_frames = [cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) for frame in frames]

    dense_flow = []

    for i in range(1, len(gray_frames)):
        prev_frame = gray_frames[i-1]
        cur_frame = gray_frames[i]

        flow = cv2.calcOpticalFlowFarneback(prev_frame, cur_frame, None, 0.5, 3, 15, 3, 5, 1.2, cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        flow[..., 0] -= np.mean(flow[..., 0])
        flow[..., 1] -= np.mean(flow[..., 1])
        # normalize
        flow[..., 0] = cv2.normalize(flow[..., 0], None, 0, 255, cv2.NORM_MINMAX)
        flow[..., 1] = cv2.normalize(flow[..., 1], None, 0, 255, cv2.NORM_MINMAX)

        dense_flow.append(flow)
    # Padding the last frame as empty array
    dense_flow.append(np.zeros((224, 224, 2)))

    return np.array(dense_flow, dtype=np.float32)


In [ ]:
def mypredict(model, video_path):
  testing_frame = video2frame(video_path)
  coba_frame = testing_frame[:, :, :, :3]
  coba_frame = coba_frame//255.0
  coba_frame = np.reshape(coba_frame, (1, 20, 224, 224, 3))
  pred = model_no_pretrain.predict(coba_frame)
  predicted_class = np.argmax(pred, axis=-1)
  return pred, predicted_class

In [ ]:
coba2_pred, coba2_class = mypredict(model_no_pretrain, "/content/t_n006_converted.avi")
coba2_class

1/1 [==============================] - 10s 10s/step


array([0])

In [ ]:
coba2_pred, coba2_class = mypredict(model_no_pretrain, "/content/t_n006_converted.avi")
coba2_class

1/1 [==============================] - 7s 7s/step


array([0])